In [1]:
!pwd

/home/yutankim/Desktop/EfficientDet_TensorFlow2


In [4]:
import tensorflow as tf

from configuration import Config
from core.efficientdet import EfficientDet, PostProcessing
from data.dataloader import DataLoader
|
def print_model_summary(network):
    sample_inputs = tf.random.normal(shape=(Config.batch_size, Config.get_image_size()[0], Config.get_image_size()[1], Config.image_channels))
    sample_outputs = network(sample_inputs, training=True)
    network.summary()

#모델 구조
model = EfficientDet()
print_model_summary(model)

#weight 불러오기
load_weights_from_epoch = Config.load_weights_from_epoch_quan
model.load_weights(filepath=Config.save_model_dir+"epoch-{}".format(load_weights_from_epoch))
#model.load_weights(filepath=Config.save_model_dir+"saved_model")

SyntaxError: invalid syntax (1289490879.py, line 6)

In [5]:
import numpy  as np
from PIL import Image
import os

# path = './data/fire_smoke/train/JPEGImages'
# count = 0
# image_list = []
# for item in os.listdir(path):
#     imgpath = path +'/' + item
#     img = Image.open(imgpath)
#     arr = np.array(img)
#     image_list.append(arr)
#     #img.save(str(count) +  "output.png")
#     count = count +1
# print(np.array(image_list).shape)

# for image in tf.data.Dataset.from_tensor_slices(image_list).batch(1).take(500):
#     image = tf.image.resize(image, (640, 640))
#     print(image.shape)

In [6]:
# 여기 부분, 기존 코드의 data_preprocesssing 과 동일하게 맞추기
# 배치사이즈 ( 32 ) 에 동일하게 맞춰서 넣어줘야한다.

# def representative_data_gen():
#     dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'/*')
#     for i in range(100):
#         image = next(iter(dataset_list))
#         image = tf.io.read_file(image)
#         image = tf.io.decode_jpeg(image, channels=3)
#         image = tf.image.resize(image, (512,512))
#         #image = tf.cast(image / 255., tf.float32)
#         image = tf.expand_dims(image, 0)
#         print(image.shape)
# # Model has only one input so each data point has one element
#         yield [image]
    
# def representative_data_gen():
#     dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'/*')
#     image_data = []
#     for dir in dataset_list:
        
    
#     for j in range(10):
#         image_temp = [ ]
#         for temp_dir in dataset_list.take(32):
#             image = tf.io.read_file(temp_dir)
#             image = tf.io.decode_jpeg(image, channels=3)
#             image = tf.image.resize(image, (512,512))
#             #image = tf.cast(image / 255., tf.float32)
#             image = tf.expand_dims(image, 0)
#             image_temp.append(image)
#     # Model has only one input so each data point has one element
#         yield [image_temp]
        
def representative_data_gen():
    path = './data/fire_smoke/train/JPEGImages'
    count = 0
    image_list = []
    for item in os.listdir(path):
        imgpath = path +'/' + item
        img = Image.open(imgpath)
        arr = np.array(img)
        image_list.append(arr)
        #img.save(str(count) +  "output.png")
        count = count +1

    for image in tf.data.Dataset.from_tensor_slices(image_list).batch(1).take(500):
        image = tf.image.resize(image, (640, 640))
        yield [image]
            
#     #### 수정좀 해보기
# def representative_data_gen():
#   dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'\\*')
#   for i in range(100):
#     image = next(iter(dataset_list))
#     image = tf.io.read_file(image)
#     image = tf.io.decode_jpeg(image, channels=3)
#     image = tf.image.resize(image, (512,512))
#     # image = tf.cast(image / 255., tf.float32)
#     image = tf.expand_dims(image, 0)
# # Model has only one input so each data point has one element
#     yield [image]

In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.experimental_new_quantizer = True
converter.inference_input_type = tf.uint8
#converter.inference_output_type = tf.uint8
converter.inference_output_type = tf.float32

tflite_model_quant = converter.convert()

2021-10-30 14:42:16.652114: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2021-10-30 14:42:16.652229: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-10-30 14:42:16.654754: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22854 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:18:00.0, compute capability: 7.5
2021-10-30 14:42:16.701387: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.008ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2021-10-30 14:42:20.510731: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2021-10-30 14:42:20.510774: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuf

In [10]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)

In [11]:
#interpreter.resize_tensor_input(0, [ 32 , 512, 512,   3])
interpreter.allocate_tensors()
for item in interpreter.get_tensor_details():
    for key in item.keys():
        print("%s : %s" % (key, item[key]))
    print("")

name : input_1
index : 0
shape : [  1 640 640   3]
shape_signature : [  1 640 640   3]
dtype : <class 'numpy.uint8'>
quantization : (1.0, 0)
quantization_parameters : {'scales': array([1.], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/efficient_net/sequential/mb_conv/se_block/global_average_pooling2d/Mean/reduction_indices
index : 1
shape : [2]
shape_signature : [2]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/bi_fpn/bi_fpn_module/up_sampling2d/mul
index : 2
shape : [2]
shape_signature : [2]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/bi_fpn

      dtype=int32), 'quantized_dimension': 3}
sparsity_parameters : {}

name : efficient_det/efficient_net/sequential_2/mb_conv_6/batch_normalization_20/FusedBatchNormV3
index : 74
shape : [240]
shape_signature : [240]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([1.5606003 , 2.989298  , 2.6054115 , 2.234505  , 3.3518841 ,
       3.152118  , 1.5507358 , 2.0916286 , 1.0557086 , 1.0810654 ,
       2.3221688 , 1.3702577 , 1.1811639 , 2.085288  , 0.8239636 ,
       2.0006752 , 2.3346615 , 2.7779973 , 0.7628082 , 1.8137408 ,
       1.9520599 , 2.1571007 , 1.4934297 , 3.544844  , 3.5372176 ,
       0.9973006 , 1.5243751 , 3.8777747 , 2.6584523 , 2.0739787 ,
       1.3997208 , 1.6415335 , 0.67170066, 1.5111675 , 2.5259955 ,
       1.841372  , 1.9365385 , 1.5151745 , 2.8113067 , 1.2188433 ,
       3.1828456 , 1.4553884 , 0.9638355 , 3.3210564 , 1.5642699 ,
       1.4342484 , 2.450242  , 2.3124542 , 1.0650285 , 1.1105778 ,
       1.2498893 , 1

shape : [120]
shape_signature : [120]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([0.61243093, 0.620838  , 0.64060086, 0.615338  , 0.7017441 ,
       0.61428326, 0.6459373 , 0.6547301 , 0.61425537, 0.65966076,
       0.70094174, 0.64537096, 0.7160979 , 0.6502946 , 0.62025845,
       0.6594608 , 0.6584609 , 0.67702866, 0.64403975, 0.6980522 ,
       0.65465134, 0.6390048 , 0.6895286 , 0.69547933, 0.6132795 ,
       0.6338375 , 0.70346856, 0.6794549 , 0.73792166, 0.7129325 ,
       0.65079534, 0.6692276 , 0.65858203, 0.70656747, 0.67803544,
       0.6238068 , 0.69615555, 0.67140985, 0.6088169 , 0.6259682 ,
       0.61542356, 0.6462447 , 0.6846131 , 0.6680346 , 0.6592842 ,
       0.6589607 , 0.6102467 , 0.6252597 , 0.6017886 , 0.68854713,
       0.67008984, 0.6516246 , 0.6384402 , 0.63351756, 0.6387815 ,
       0.6821524 , 0.708406  , 0.61579454, 0.6222661 , 0.6167447 ,
       0.75025445, 0.70443773, 0.6309945 , 0.64834964, 0.63917047,


name : efficient_det/efficient_net/sequential_4/mb_conv_13/se_block_13/conv2d_55/Conv2D
index : 154
shape : [672   1   1 168]
shape_signature : [672   1   1 168]
dtype : <class 'numpy.int8'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([0.00079482, 0.00069421, 0.0007228 , 0.00080559, 0.00076424,
       0.00067092, 0.00078658, 0.0008018 , 0.00089679, 0.00084792,
       0.00076041, 0.00074757, 0.00069719, 0.00080635, 0.00073411,
       0.00071506, 0.00071056, 0.0007469 , 0.00076828, 0.00076749,
       0.00067991, 0.00070701, 0.00080684, 0.00073863, 0.00081669,
       0.00073593, 0.00072221, 0.00074156, 0.00071712, 0.00066566,
       0.00071356, 0.00069916, 0.00067797, 0.00069766, 0.00073028,
       0.00072637, 0.00083804, 0.00068613, 0.00085878, 0.00077012,
       0.00074942, 0.00077552, 0.00066175, 0.00068901, 0.0008062 ,
       0.00070961, 0.00074099, 0.00071905, 0.00069906, 0.00070982,
       0.00076807, 0.00075528, 0.00067439, 0.00075622, 0.00077955,
       0.00

quantization_parameters : {'scales': array([2.5049772, 3.1943572, 2.0650306, 3.4212713, 2.5260162, 3.3184898,
       2.624547 , 2.8199315, 2.7051642, 3.0922341, 3.1789424, 4.212254 ,
       2.3959794, 2.1928627, 3.127503 , 2.8606174, 4.2829795, 2.885523 ,
       2.7476192, 3.0200398, 3.5768046, 2.9259758, 2.1800132, 2.7794206,
       2.6792467, 2.5659418, 3.3232255, 3.15903  , 2.555445 , 1.9613342,
       4.131738 , 2.4488838, 3.6503613, 3.759089 , 2.645875 , 2.6466022,
       3.0078404, 2.9112234, 3.288388 , 2.8625367, 3.2296777, 3.5717466,
       3.0434308, 2.6847854, 2.7811759, 3.0284479, 2.9164207, 2.017815 ,
       2.1412902, 2.412264 , 2.422335 , 2.763659 , 4.0924206, 2.7773871,
       3.0554454, 3.0089881, 2.5813282, 2.9779496, 2.6252606, 2.4508867,
       2.7576637, 2.4369786, 3.3494878, 2.7592895, 1.8841288, 3.1541216,
       3.0036364, 2.3639817, 3.1099837, 3.7716987, 2.4980724, 2.448009 ,
       2.4387112, 2.6892755, 2.581537 , 2.978746 , 2.2749794, 2.8884208,
       2.43589

...

shape : [ 1 40 40 80]
shape_signature : [ 1 40 40 80]
dtype : <class 'numpy.int8'>
quantization : (2633.38232421875, 20)
quantization_parameters : {'scales': array([2633.3823], dtype=float32), 'zero_points': array([20], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/efficient_net/sequential_3/mb_conv_11/add_7/add
index : 635
shape : [ 1 40 40 80]
shape_signature : [ 1 40 40 80]
dtype : <class 'numpy.int8'>
quantization : (4646.28955078125, 11)
quantization_parameters : {'scales': array([4646.2896], dtype=float32), 'zero_points': array([11], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/bi_fpn/conv_norm_act_1/batch_normalization_71/FusedBatchNormV3;efficient_det/bi_fpn/conv_norm_act_1/conv2d_94/BiasAdd/ReadVariableOp/resource;efficient_det/bi_fpn/conv_norm_act_1/conv2d_94/BiasAdd;efficient_det/box_class_predict/conv_norm_act_10/conv2d_103/Conv2D_4;efficient_det/bi_fpn/conv_norm_act_1/conv2d_94/Conv2D
index 

quantization_parameters : {'scales': array([0.00390625], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/efficient_net/sequential_5/mb_conv_20/se_block_20/mul_1
index : 774
shape : [   1   10   10 1152]
shape_signature : [   1   10   10 1152]
dtype : <class 'numpy.int8'>
quantization : (6410.30322265625, 0)
quantization_parameters : {'scales': array([6410.303], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/efficient_net/sequential_5/mb_conv_20/Sigmoid_1
index : 775
shape : [   1   10   10 1152]
shape_signature : [   1   10   10 1152]
dtype : <class 'numpy.int8'>
quantization : (0.00390625, -128)
quantization_parameters : {'scales': array([0.00390625], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/efficient_net/sequential_5/mb_conv_20/mul

sparsity_parameters : {}

name : efficient_det/bi_fpn/bi_fpn_module_1/weighted_feature_fusion_13/separable_conv_norm_act_13/mul
index : 943
shape : [ 1 20 20 88]
shape_signature : [ 1 20 20 88]
dtype : <class 'numpy.int8'>
quantization : (1770.1539306640625, -128)
quantization_parameters : {'scales': array([1770.1539], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/bi_fpn/bi_fpn_module_1/max_pooling2d_6/MaxPool
index : 944
shape : [ 1 10 10 88]
shape_signature : [ 1 10 10 88]
dtype : <class 'numpy.int8'>
quantization : (1770.1539306640625, -128)
quantization_parameters : {'scales': array([1770.1539], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/bi_fpn/bi_fpn_module_1/weighted_feature_fusion_14/mul_2
index : 945
shape : [ 1 10 10 88]
shape_signature : [ 1 10 10 88]
dtype : <class 'numpy.int8'>
quantization : (640.1657

shape : [ 1 10 10 88]
shape_signature : [ 1 10 10 88]
dtype : <class 'numpy.int8'>
quantization : (501.89227294921875, -4)
quantization_parameters : {'scales': array([501.89227], dtype=float32), 'zero_points': array([-4], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/bi_fpn/bi_fpn_module_2/weighted_feature_fusion_22/separable_conv_norm_act_22/batch_normalization_97/FusedBatchNormV3;efficient_det/bi_fpn/bi_fpn_module_2/weighted_feature_fusion_22/separable_conv_norm_act_22/separable_conv2d_22/BiasAdd/ReadVariableOp/resource;efficient_det/bi_fpn/bi_fpn_module_2/weighted_feature_fusion_22/separable_conv_norm_act_22/separable_conv2d_22/BiasAdd;efficient_det/box_class_predict/conv_norm_act_10/conv2d_103/Conv2D_4;efficient_det/bi_fpn/bi_fpn_module_2/weighted_feature_fusion_22/separable_conv_norm_act_22/separable_conv2d_22/separable_conv2d
index : 1094
shape : [ 1 10 10 88]
shape_signature : [ 1 10 10 88]
dtype : <class 'numpy.int8'>
quantization : (611

shape : [2200]
shape_signature : [2200]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : 
index : 1251
shape : [5]
shape_signature : [5]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : 
index : 1252
shape : [1]
shape_signature : [1]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : 
index : 1253
shape : [8800]
shape_signature : [8800]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantize

In [12]:
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.float32'>


In [13]:
with open('./tflite_model_quant_D1_SGD.tflite', 'wb') as f:
  f.write(tflite_model_quant)

In [14]:
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
print(input_details)
print("-"*50)
print(output_details)

{'name': 'input_1', 'index': 0, 'shape': array([  1, 640, 640,   3], dtype=int32), 'shape_signature': array([  1, 640, 640,   3], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (1.0, 0), 'quantization_parameters': {'scales': array([1.], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
--------------------------------------------------
{'name': 'Identity', 'index': 1157, 'shape': array([    1, 76725,     5], dtype=int32), 'shape_signature': array([    1, 76725,     5], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
